### Resources:
* [D-separation steps](http://web.mit.edu/jmn/www/6.034/d-separation.pdf)
* [bnlearn datasets](https://www.bnlearn.com/bnrepository/)
* [Daggity paper](https://academic.oup.com/ije/article/45/6/1887/2907796)
* [Milestone](https://gitlab.pnnl.gov/y0-scuc/y0-archive/-/milestones/4)
* [pgmpy](https://github.com/pgmpy/pgmpy/blob/dev/examples/Creating%20a%20Discrete%20Bayesian%20Network.ipynb)  -- get_independencies

### TODO:
* Break into module, write proper unit tests
* What about conditional independencies?  Currently only covers independencies.  
   * Can I just enumerate tripples and check 'A,B given=["C"]'?  
   * Would then check combinations-pairs + combinations-givens 
   * Up to how many givens do you check?

In [1]:
from y0.algorithm import falsification
import y0.examples
from y0.dsl import Variable
import copy
import networkx as nx
import pandas as pd

import seaborn as sns
from itertools import chain
from tqdm import tqdm

%matplotlib inline

In [2]:
asia_G = y0.examples.asia_example.graph
asia_df = y0.examples.asia_example.data

asia_G.draw()

In [3]:
asia_df

In [4]:
issues = falsification.get_graph_falsifications(asia_G, asia_df, max_given=2, verbose=True)

In [5]:
issues.evidence

In [6]:
# Should be issues (Lung, Either) AT LEAST.
# .  Seeing issues (Lung, Xray) as well.  Is that all?
# TODO: Check d-separations by hand...
asia_G2 = copy.deepcopy(asia_G)
asia_G2.draw()

In [7]:
asia_G2.directed.remove_edge(
    Variable("Lung"), Variable("Either")
)  # delete_diedge("Lung", "Either")
issues = falsification.get_graph_falsifications(asia_G2, asia_df)

In [8]:
issues.failures

In [9]:
issues.evidence